## The Cold Start Problem 

The colaborative filtering method discussed in class does not address the problem of new user or new movies. What prediction would you use in these cases:

* A new user but a known movie
* A new movie and a known user
* A new user and new movie

**A new user but a known movie**
* Average value given by other users if the new user never rated any movie.
* Average value given by users in similiar demography/ if we have that information.
* Use implicit rating to augment the preference of the new user and suggest based on that.

**A new movie and a known user**
* Cluster movies with similar content as new movie. Take weighted average of ratings based on similarity between new movie and similar movies liked by user.
* Find the ratings given by users similar to known user. Take weighted average of ratings based on similarity between know users and others for given movie.

**A new user and new movie**
* Start with what is trending in the user demography, if we dont have that information, use the global trends. 
* We could also start collecting the implicit ratings (views/clicks) to tune our recommendation.

## Matrix Factorization with bias
We want to extend the Matrix Factorization model discussed in class to add a "bias" parameter for each user and another "bias" parameter for each movie.  For the problem in class we had the parameters matrix $U$ and $V$, we are adding $u^0$ which is a vector of dimension $n_u$ and $v^0$ which is a vector of dimension $n_m$. The equations

$$\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $$ 
 
(a) How many weights (parameters) are we fitting for this problem?

(b) Write the gradient descent equations for this problem.

(a) $$n_u\cdot(k+1) + n_m\cdot(k+1)$$
(b) Equation : $$Loss(E) = [(\hat{Y} -  U\cdot V^T - U_{0} - V_{0}^T)^2 * R]/ N$$

* $$ dE / du_{0i} =  -2/N  \cdot np.sum([\hat{Y} -  U\cdot V^T - U_{0} - V_{0}^T]*R, axis = 0)$$    $$u_{0i} = u_{0i} - \eta * dE / du_{0i}$$

 $$ dE / dv_{0j} =  -2/N  \cdot np.sum([\hat{Y} -  U\cdot V^T - U_{0} - V_{0}^T]*R, axis = 1)^T$$    $$v_{0j} = v_{0j} - \eta * dE / dv_{0i}$$

 $$ dE/ dV = -2/N  \cdot np.sum(([\hat{Y} -  U\cdot V^T - U_{0} - V_{0}^T]*R)\cdot U))  $$ $$V = V - \eta * dE / dV$$

 $$ dE/ dU = -2/N  \cdot np.sum(([\hat{Y} -  U\cdot V^T - U_{0} - V_{0}^T]*R)^T\cdot V))  $$  $$U = U - \eta * dE / dU$$

# Collaborative Filtering with Gradient Descent 

In this part of the assignment you will build a collaborative filtering model to predict netflix ratings.  This assignment will step you through how to do this using stochastic gradient descent. The data for this assignment is ...

**Instructions:**
- Do not use loops (for/while) in your code, unless the instructions explicitly ask you to do so.
- DO NOT change paths (-3 points)
- DO NOT submit data to github (-2 points)

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Encoding rating data
    - Initializing parameters
    - Calculating the cost function
    - Calculating gradient
    - Using an optimization algorithm (gradient descent) 
    - Predicting on new data
- Putting it all together.

In [1]:
import numpy as np
import pandas as pd


## Encoding rating data
Here are our very small subset of fake data to get us started.

In [2]:
# The first row says that user 11 reated movie 1 with a score of 4
!cat tiny_training2.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [3]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [4]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    _, df['userId'],num_users = proc_col(df['userId'])
    _, df['movieId'],num_movies = proc_col(df['movieId'])
    
    return df, num_users, num_movies

In [5]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)

In [6]:
df

,userId,movieId,rating
0,0,0,4
1,0,1,5
2,1,1,5
3,1,2,3
4,2,0,4
5,2,1,4
6,3,0,5
7,3,3,2
8,4,0,1
9,4,3,4


In [7]:
assert(num_users == 7)

In [8]:
assert(num_movies == 4)

In [9]:
np.testing.assert_equal(df["userId"].values, np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 6, 6]))

## Initializing parameters

In [10]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[3.55790894, 4.69774849, 0.92361109, 1.58739544, 3.00593239],
       [4.69774849, 7.44656163, 1.18135616, 2.64524868, 4.74559066],
       [0.92361109, 1.18135616, 0.24548062, 0.34025121, 0.69616965],
       [1.58739544, 2.64524868, 0.34025121, 1.61561   , 2.41361975],
       [3.00593239, 4.74559066, 0.69616965, 2.41361975, 3.82505541],
       [2.02000808, 3.29656257, 0.43174569, 2.065911  , 3.07264619],
       [2.07691001, 3.02887291, 0.53270924, 1.02482544, 1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [11]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [12]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [13]:
print(Y)

  (0, 0)	4
  (2, 0)	4
  (3, 0)	5
  (4, 0)	1
  (0, 1)	5
  (1, 1)	5
  (2, 1)	4
  (6, 1)	1
  (1, 2)	3
  (3, 3)	2
  (4, 3)	4
  (5, 3)	5
  (6, 3)	3


## Predicting Ratings

In [14]:
def predict(df, emb_user, emb_movie):
    """ This function computes df["prediction"] without doing (U*V^T).
    
    Compute df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """
    df['prediction'] = np.multiply(emb_user[df['userId']] , emb_movie[df['movieId']]).sum(axis = 1)
    return df

In [15]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
df = predict(df, emb_user, emb_movie)
assert(df["prediction"][12] == 3.0)

In [16]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
df = predict(df, emb_user, emb_movie)
pred_12 = df["prediction"][12]
assert(np.around(pred_12, decimals=2)== 2.05)

## Calculating the cost function

In [17]:
# Use vectorized computation for this function. No loops!
# Hint: use predict function
def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction using the predict function.
    Prediction for user i and movie j is emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    df = predict(df, emb_user, emb_movie)
    error = np.mean(np.square(df['prediction'] - df['rating']))
    
    return error

In [18]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

In [19]:
emb_user = create_embedings(num_users, K=4)
emb_movie = create_embedings(num_movies, K=4)
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 4.36)

## Calculating gradient

In [20]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [21]:
def gradient(df, emb_user, emb_movie):
    """ Computes the gradient.
    
    Hint: First compute df["prediction"]. Then use df2matrix
    to get a sparse matrix Y and Y_hat.
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    df = predict(df, emb_user, emb_movie)
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="rating")
    R = Y.sign().todense()
    Y_hat = df2matrix(df,emb_user.shape[0], emb_movie.shape[0], column_name='prediction').todense()

    diff = np.multiply(Y.todense(),R) - Y_hat
    d_emb_user = -2*(1/len(Y.data))*(np.dot(diff, emb_movie))
    d_emb_movie = -2*(1/len(Y.data))*(np.dot(diff.transpose(), emb_user))
                                   
    
    return d_emb_user,d_emb_movie

In [22]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
grad_user, grad_movie = gradient(df, emb_user, emb_movie)

In [23]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [24]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Using gradient descent with momentum

In [25]:
# you can use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    cost_train = cost(df, emb_user, emb_movie)
    cost_val = None
    if df_val is not None:
        cost_val = cost(df_val, emb_user, emb_movie)
    print(cost_train, cost_val)
    grad_user_mom, grad_movie_mom = gradient(df,emb_user,emb_movie)
    emb_user = emb_user - learning_rate  * grad_user_mom 
    emb_movie = emb_movie - learning_rate * grad_movie_mom
    

    for i in range(iterations-1):

        grad_user,grad_movie = gradient(df,emb_user,emb_movie)
        grad_user_mom = .9*grad_user_mom+.1*grad_user
        grad_movie_mom = .9*grad_movie_mom + .1*grad_movie
        emb_user = emb_user - learning_rate  * grad_user_mom 
        emb_movie = emb_movie - learning_rate * grad_movie_mom
        cost_train = cost(df, emb_user, emb_movie)
        if df_val is not None:
            cost_val = cost(df_val, emb_user, emb_movie)

        if i%50 == 0:
            print(cost_train, cost_val)
    return emb_user, emb_movie

In [26]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

4.765335560733245 None
4.542728284061158 None
1.650386132464952 None
0.9595365363702619 None
0.6910673020186648 None


In [27]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.53)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [28]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    user2indx, _, _ = proc_col(df_train['userId'])
    movie2indx, _, _ = proc_col(df_train['movieId'])
    valCol_user = df_val['userId'].apply(lambda x: user2indx[x] if x in user2indx else -1 )
    valCol_movie = df_val['movieId'].apply(lambda x: movie2indx[x] if x in movie2indx else -1 )
    df_val['userId'] = valCol_user
    df_val['movieId'] = valCol_movie
    df_val = df_val[(df_val['userId'] != -1) & (df_val['movieId'] != -1)]
    
    return df_val

In [29]:
df_t = pd.read_csv("tiny_training2.csv")
df_v = pd.read_csv("tiny_val2.csv")
df_v = encode_new_data(df_v, df_t)

In [30]:
assert(len(df_v.userId.unique())==2)

In [31]:
assert(len(df_v) == 2)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [32]:
# Don't change this path use a simlink if you have the data somewhere else
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20386 19591


In [33]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=200, learning_rate=1, df_val=df_val)

12.114879435617219 12.201680645501428
12.01726704778792 12.106537746029804
9.36601375978891 9.494304769287112
6.593882562086153 6.7264322075991885
4.749827241675142 4.874050651928023


In [34]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

3.756714365159779 3.8685464737636472


Please check that when you run gradient descent for 2000 iterations. `val_mse` should be around 0.855

In [35]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=1) == 3.8)

In [ ]:
# K = 50
# emb_user = create_embedings(num_users, K)
# emb_movie = create_embedings(num_movies, K)
# emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000, learning_rate=1, df_val=df_val)

In [ ]:
# train_mse = cost(df_train, emb_user, emb_movie)
# assert(np.around(train_mse, decimals=3) == 0.856)